# ===========================================================

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
import tensorflow.keras.regularizers as regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
import tensorflow_addons as tfa
from datetime import datetime
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(100)
import random
random.seed(100)
tf.random.set_seed(100)

class CNN:
    def __init__(self,dims,w2v_path,max_seq_len=20,batch_size=128,epochs=20):
        self.dims = dims
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.epochs = epochs
        with open(w2v_path, 'rb') as f:            
            self.w2v = pickle.load(f)
        self.model = None        
        self.label_mapping = None
        self.n_classes = None
        self.history = None
        self.metrics = None #[tf.keras.metrics.AUC(name='auc'), tfa.metrics.F1Score(self.n_classes, average='weighted', name='f1_score'), 'accuracy']
        log_dir = f"logs/fit/run_only_once" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
        decay_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001 ,min_lr=0.00001)
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
        self.callbacks = [tensorboard_callback, decay_rate, early_stopping]


    def build_cnn(self):
        if self.n_classes > 2:
            loss = 'categorical_crossentropy'
            activation = 'softmax'
        else:
            loss = 'binary_crossentropy'
            activation = 'sigmoid'

        input_layer = layers.Input(shape=(self.max_seq_len, 300))
        conv1_1 = layers.Conv1D(128, 4, activation='relu', padding='same')(input_layer)
        conv1_2 = layers.Conv1D(128, 5, activation='relu', padding='same')(conv1_1)
        #conv1_3 = layers.Conv1D(128, 5, activation='relu', padding='same')(conv1_2)
        conv_out = layers.Concatenate(axis=1)([conv1_1, conv1_2])

        dropout_rate = 0.5
        dropout_out1 = layers.Dropout(dropout_rate)(conv_out)

        pool_out = layers.MaxPool1D(pool_size=self.max_seq_len, padding='valid')(dropout_out1)
        flatten_out = layers.Flatten()(pool_out)
        dropout_out2 = layers.Dropout(dropout_rate)(flatten_out)
        dense_out = layers.Dense(self.n_classes, activation=activation, kernel_regularizer=regularizers.L2(0.001))(dropout_out2)
        
        self.metrics = [tf.keras.metrics.AUC(name='auc'), tfa.metrics.F1Score(self.n_classes, average='weighted', name='f1_score'), 'accuracy']
        cnn_model = Model(inputs=input_layer, outputs=dense_out)
        cnn_model.compile(optimizer='adam', loss=loss, metrics=self.metrics)
        #cnn_model.summary()
        self.model = cnn_model
        
    def insert_values(self,train_path,test_path):    
        def insert(df):
            
            # initialize x self.and y matrices
            num_lines = len(df)
            self.n_classes = df['class'].nunique()          
            x_matrix = np.zeros((num_lines, self.max_seq_len ,300))
            y_matrix = np.zeros((num_lines, self.n_classes))


            # insert values
            for i, row in df.iterrows():
                label = row[0]
                sentence = row[1]
                if isinstance(sentence, str):
                    words = sentence.split()[:self.max_seq_len]
                    for j, word in enumerate(words):
                        if word in self.w2v:
                            x_matrix[i, j, :] = self.w2v[word]
                else:
                    continue        
                y_matrix[i,label] = 1.0    

            return x_matrix,y_matrix
        
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)

        self.n_classes = train_df['class'].nunique()        
        unique_classes = train_df['class'].unique()
        labels_map = dict(zip(unique_classes, range(self.n_classes)))

        train_df['class'] = train_df['class'].map(labels_map)
        test_df['class'] = test_df['class'].map(labels_map)

        train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=100)
        print(f'Train size: {len(train_df)}\nValidation size: {len(val_df)}\nTest size: {len(test_df)}')

        train_df = train_df.sample(frac=1).reset_index(drop=True)
        test_df = test_df.sample(frac=1).reset_index(drop=True)
        val_df = val_df.sample(frac=1).reset_index(drop=True)

        train_x, train_y = insert(train_df)
        test_x, test_y = insert(test_df)
        val_x, val_y = insert(val_df)

        return train_x, train_y, test_x, test_y, val_x, val_y, self.n_classes          

    def fit(self,train_x, train_y,  val_x, val_y):
        self.metrics = [tf.keras.metrics.AUC(name='auc'), tfa.metrics.F1Score(self.n_classes, average='weighted', name='f1_score'), 'accuracy']
        self.build_cnn()  
        self.history = self.model.fit(train_x, train_y, batch_size=self.batch_size, epochs=self.epochs, validation_data=(val_x, val_y), callbacks=self.callbacks)
        return self.history
    def evaluate(self,test_x, test_y):
        return self.model.evaluate(test_x, test_y,return_dict=True)

    def run_n_times(self,train_x, train_y, test_x, test_y, val_x, val_y, n=3):
        hist_dict = {}
        res_dict = {}
        for i in range(n):
            print(f'Run {i+1} of {n}')
            hist_dict[i+1] = self.fit(train_x, train_y, val_x, val_y)
            res_dict[i+1] = self.evaluate(test_x, test_y)
            self.model.set_weights([np.zeros(w.shape) for w in self.model.get_weights()])
            # self.fit(train_x, train_y, val_x, val_y)
            # self.evaluate(test_x, test_y)
        avg_dict = {metric: round(sum(values[metric] for values in res_dict.values()) / len(res_dict), 4)  for metric in res_dict[1].keys()}
        return hist_dict, res_dict, avg_dict

    # save the results and best model
    def save_results(self, hist_dict, res_dict, avg_dict, model_name):
        # save the results
        with open(f'{model_name}_results.json', 'w') as f:
            json.dump(res_dict, f)
        with open(f'{model_name}_avg_results.json', 'w') as f:
            json.dump(avg_dict, f)
        # save the best model
        self.model.save(model_name)
    

In [2]:
train_path  = 'data/original/cr/train.csv'
test_path   = 'data/original/cr/test.csv'
w2v_path = 'w2v.pkl'
max_seq_len = 32
batch_size = 128
epochs = 30

In [3]:
model = CNN(dims=300, max_seq_len=max_seq_len, batch_size=batch_size, epochs=epochs, w2v_path=w2v_path)

In [4]:
train_x, train_y, test_x, test_y, val_x, val_y, n_classes = model.insert_values(train_path,test_path)

Train size: 3660
Validation size: 407
Test size: 451


In [5]:
# his = model.fit(train_x, train_y, val_x, val_y)
# res = model.evaluate(test_x, test_y)
his,res,avg = model.run_n_times(train_x, train_y, test_x, test_y, val_x, val_y, n=3)

Run 1 of 3
Epoch 1/30
29/29 [==============================] - 5s 95ms/step - loss: 0.6960 - auc: 0.6833 - f1_score: 0.6430 - accuracy: 0.6730 - val_loss: 0.5441 - val_auc: 0.8483 - val_f1_score: 0.7486 - val_accuracy: 0.7666 - lr: 0.0010
Epoch 2/30
29/29 [==============================] - 2s 78ms/step - loss: 0.4984 - auc: 0.8374 - f1_score: 0.7628 - accuracy: 0.7694 - val_loss: 0.4635 - val_auc: 0.9036 - val_f1_score: 0.8299 - val_accuracy: 0.8305 - lr: 0.0010
Epoch 3/30
29/29 [==============================] - 2s 76ms/step - loss: 0.4249 - auc: 0.8871 - f1_score: 0.8049 - accuracy: 0.8071 - val_loss: 0.4254 - val_auc: 0.9188 - val_f1_score: 0.8425 - val_accuracy: 0.8428 - lr: 0.0010
Epoch 4/30
29/29 [==============================] - 2s 76ms/step - loss: 0.3733 - auc: 0.9156 - f1_score: 0.8373 - accuracy: 0.8380 - val_loss: 0.3946 - val_auc: 0.9288 - val_f1_score: 0.8519 - val_accuracy: 0.8526 - lr: 0.0010
Epoch 5/30
29/29 [==============================] - 2s 77ms/step - loss: 0.32